<h1>Calling Redit API for Data Collection</h1>

In [1]:
import datetime

import praw
import pandas as pd

from keys import client_id, client_secret

<h3>1. Collecting the posts for topic- Donald Trump</h3>

Declaring the Reddit Instance

In [2]:

reddit = praw.Reddit( client_id='BNyCLXvMfBaDuA',
                      client_secret='_ufZ8F5WpKUP3gn5QfDjzsF1jkg',
                      user_agent='android:my_app:v1 (by /u/uditsharma16)')

API call and building the DataFrame

In [3]:
tesla = reddit.subreddit('donaldtrump')

#Gathering the top 500 posts, with their title, url, body, upvotes, timestamp, and an index that serves as a key between the
#posts and the comments we collect later
posts = []
for index, post in enumerate(tesla.top(limit=500)):
    posts.append([post.title, "https://www.reddit.com" + post.permalink, post.selftext, post.score, post.created_utc, index])

#Converting into DataFrame
posts = pd.DataFrame(posts, columns=['Title', 'URL', 'Body', 'Upvotes', 'Time', 'Key'])
#Changing from utc time to standard timestamp
posts.Time = posts.Time.apply(lambda x: pd.to_datetime(datetime.datetime.fromtimestamp(x)))

#The first post is a sticky, so we can drop it
posts = posts.iloc[1:]

In [4]:
posts.head(3)

,Title,URL,Body,Upvotes,Time,Key
1,"Roads? Where we’re going, we don’t need roads.",https://www.reddit.com/r/donaldtrump/comments/...,,1457,2020-09-12 01:22:07,1
2,BRILLIANT,https://www.reddit.com/r/donaldtrump/comments/...,,1425,2020-09-17 05:30:01,2
3,All it took was one Trump supporting black man...,https://www.reddit.com/r/donaldtrump/comments/...,,1403,2020-08-01 05:28:10,3


In [5]:
posts.shape

(499, 6)

<h3>2. Collecting the comments for each of post</h3>

Defining function to get all comments from a post

In [6]:
def collect_replies(key, url):
    ''' 
    params pandas series row: each row of the dataframe we built above in the form of a panda series
    Returns a pandas DataFrame, where each row represents an individual comment
    '''
    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=None)
    comment_queue = submission.comments[:] 

    table = {'Reply':[], 'Upvote':[], 'Time':[], 'Key':[]}

    while comment_queue:
        comment = comment_queue.pop(0)
        table['Reply'].append(comment.body)
        table['Time'].append(comment.created_utc)
        table['Upvote'].append(comment.score)
        table['Key'].append(key)
        comment_queue.extend(comment.replies)
    
    return pd.DataFrame.from_dict(table)

Creating DataFrame for Commments

In [7]:
#Let us first generate a list of tupules that contains the key and url for each row - the first value of the tupule is key,
#and the second value is url
keys = posts.Key.tolist()
urls = posts.URL.tolist()
tupules = list(zip(keys, urls))

#Now we generate our comments dataframe using list comprehensions!
comments = pd.concat([collect_replies(x[0], x[1]) for x in tupules])

In [8]:
#Again, converting the timestamp from utc to a standard format
comments.Time = comments.Time.apply(lambda x: pd.to_datetime(datetime.datetime.fromtimestamp(x)))

In [9]:
comments.head(3)

,Reply,Upvote,Time,Key
0,\nThis subreddit is a pro-Trump subreddit for ...,1,2020-09-12 01:22:08,1
1,Good one. I laughed out loud,59,2020-09-12 02:06:40,1
2,"Also butter. Got rid of the native girl, kept ...",29,2020-09-12 04:11:04,1


In [10]:
comments.shape

(65305, 4)

<h3>3. Converting the Comments and post to CSV </h3>

In [11]:
comments.to_csv(r'C:\Users\udit sharma\Desktop\MSA\sentiment-analysis-msa-phase-2\Comments1_.csv', index=False)

In [12]:
posts.to_csv(r'C:\Users\udit sharma\Desktop\MSA\sentiment-analysis-msa-phase-2\Posts1.csv', index=False)